In [ ]:
import requests as r
import pandas as pd
import json
import logging

In [ ]:
class HappyFarmer:
    def __init__(self):
        self.url = 'https://parseapi.back4app.com/'
        self.app_id = 'ASK PTCAN'
        self.key = 'ASK PTCAN'
        self.usr = 'ASK PTCAN'
        self.pwd = 'ASK PTCAN'

In [ ]:
class getRefreshToken:
    def __init__(self):
        hf = HappyFarmer()
        headers = {
            'x-parse-application-id': hf.app_id,
            'x-parse-client-key': hf.key
        }
        req = r.get(hf.url + 'login?username='+ hf.usr +'&password=' + hf.pwd + '&=', headers = headers)
        headers.update({'x-parse-session-token':req.json().get('sessionToken')})
        self.token = headers

In [ ]:
class getData:
    def __init__(self, url):
        hf = HappyFarmer()
        hdr = getRefreshToken()
        self.url = url
        self.data = r.get(hf.url + 'classes/' + url, headers = hdr.token).json()
        res = self.data.get('results')
        self.tb = pd.DataFrame(res)

In [ ]:
class postData:
    def __init__(self, url, data):
        hf = HappyFarmer()
        hdr = getRefreshToken()
        self.url = url
        self.inputs = data
        self.send = r.post(hf.url + 'classes/' + url, headers = hdr.token, json = data).json()

In [ ]:
class putData:
    def __init__(self, url, data):
        hf = HappyFarmer()
        hdr = getRefreshToken()
        self.url = url
        self.inputs = data
        self.send = r.put(hf.url + 'classes/' + url, headers = hdr.token, json = data).json()

In [ ]:
class deleteData:
    def __init__(self, url, data):
        hf = HappyFarmer()
        hdr = getRefreshToken()
        self.url = url
        self.inputs = data
        self.send = r.delete(hf.url + 'classes/' + url + '/' + data, headers = hdr.token).json()

In [ ]:
class getObjectId:
    def __init__(self, url):
        hf = HappyFarmer()
        hdr = getRefreshToken()
        self.url = url
        self.data = r.get(hf.url + 'classes/' + url, headers = hdr.token).json()
        self.oid = self.data['results'][0]['objectId']

In [ ]:
farmerAkvo = pd.read_excel('data/FP-DATA_CLEANING-12060001-20190317.xlsx',skiprows=[0])

In [ ]:
farmerAkvo = farmerAkvo.drop([0])

In [ ]:
farmerAkvo = farmerAkvo.rename(columns={'34740130|Registration Number':'Registration Number'})

In [ ]:
farmerAkvo = farmerAkvo[['Identifier','Registration Number']]

In [ ]:
farmerAkvo['Registration Number'] = farmerAkvo['Registration Number'].apply(lambda x: '0'+str(x))

In [ ]:
#farmerAkvo['ObjectID'] = farmerAkvo['Registration Number'].apply(lambda x: getObjectId('SbxFarmer?where={"RegistrationNumber":"0'+str(x)+'"}').oid)

In [ ]:
currentFarmer = pd.DataFrame(getData('SbxFarmer?limit=1000').data['results'])

In [ ]:
currentFarmer = currentFarmer[['RegistrationNumber','objectId']]

In [ ]:
farmerAkvo.shape

In [ ]:
combineId = farmerAkvo.merge(currentFarmer, left_on='Registration Number', right_on='RegistrationNumber')

In [ ]:
combineId = combineId.drop(columns=['Registration Number'])

In [ ]:
uData = combineId.to_dict('records')

In [ ]:
errorVal = []
success = 0
for idx, dt in enumerate(uData):
    hf = HappyFarmer()
    hdr = getRefreshToken()
    uri = hf.url + 'classes/' + 'SbxFarmer/'+dt['objectId']
    try:
        print(r.put(uri, headers = hdr.token, json = {'Akvo':dt['Identifier']}).json())
        success = idx
    except:
        print('ERROR')
        errorVal.append({'Identifier':dt['objectId'],'objectId':dt['objectId']})